<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural - CEIA
## Alumno: Gonzalo Gontad
## Cohorte: 8va


## Vectorización

In [1]:
import numpy as np
import pandas as pd

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [4]:
#Lista de listas de palabras
def get_docs_terms(corpus):
    return [sentence.split() for sentence in corpus]
docs_terms = get_docs_terms(corpus)
docs_terms


[['que', 'dia', 'es', 'hoy'],
 ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'],
 ['martes', 'muchas', 'gracias']]

In [5]:
def get_vocabulary(docs_terms):
    return np.array(set ([terms for doc_terms in docs_terms for terms in doc_terms]))

In [6]:
corpus_vocab = get_vocabulary(docs_terms)
corpus_vocab

array({'dia', 'que', 'martes', 'hoy', 'es', 'de', 'muchas', 'el', 'gracias'},
      dtype=object)

### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [7]:
def corpus_to_one_hot_matrix(corpus):
    docs_terms = get_docs_terms(corpus)
    corpus_vocab = get_vocabulary(docs_terms)
    return np.array([np.array([1 if term in doc_terms else 0 for term in corpus_vocab.tolist()]) for doc_terms in docs_terms])

In [8]:
one_hot_corpus = corpus_to_one_hot_matrix(corpus)
#transformo a pandas para poder ver mejor los resultados
pd.DataFrame(one_hot_corpus, columns=corpus_vocab.tolist())

,dia,que,martes,hoy,es,de,muchas,el,gracias
0,1,1,0,1,1,0,0,0,0
1,1,0,1,1,1,1,0,1,0
2,0,0,1,0,0,0,1,0,1


### 3- Vectores de frecuencia
Dada una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [9]:
def corpus_tf_matrix(corpus):
    docs_terms = get_docs_terms(corpus)
    corpus_vocab = get_vocabulary(docs_terms)
    return np.array([np.array([doc_terms.count(term) for term in corpus_vocab.tolist()]) for doc_terms in docs_terms])

In [10]:
tf_matrix = corpus_tf_matrix(corpus)
#transformo a pandas
pd.DataFrame(tf_matrix, columns=corpus_vocab.tolist())


,dia,que,martes,hoy,es,de,muchas,el,gracias
0,1,1,0,1,1,0,0,0,0
1,1,0,2,1,1,1,0,1,0
2,0,0,1,0,0,0,1,0,1


In [11]:
def corpus_freq_vector(corpus):
    one_hot = corpus_to_one_hot_matrix(corpus)
    return np.array(one_hot).sum(axis=0)    

In [12]:
freq_vector = corpus_freq_vector(corpus)
#transformo a pandas
pd.DataFrame(freq_vector, index=corpus_vocab.tolist(), columns=['Frecuencia'])

,Frecuencia
dia,2
que,1
martes,2
hoy,2
es,2
de,1
muchas,1
el,1
gracias,1


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [13]:
def tf_idf(corpus):
    tf = corpus_tf_matrix(corpus)
    freq_vector = corpus_freq_vector(corpus)
    return np.array(tf * np.log10(corpus.shape[0] / freq_vector) )

In [14]:
tf_idf_vals = tf_idf(corpus)
#transformo a pandas para poder ver mejor los resultados
pd.DataFrame(tf_idf_vals, columns=corpus_vocab.tolist())

,dia,que,martes,hoy,es,de,muchas,el,gracias
0,0.176091,0.477121,0.000000,0.176091,0.176091,0.000000,0.000000,0.000000,0.000000
1,0.176091,0.000000,0.352183,0.176091,0.176091,0.477121,0.000000,0.477121,0.000000
2,0.000000,0.000000,0.176091,0.000000,0.000000,0.000000,0.477121,0.000000,0.477121


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [15]:
def docs_comparator (doc_idx, corpus):
    tf_idf_vals = tf_idf(corpus)
    #calculo la similitud coseno entre el documento que quiero comparar y todo el corpus
    cos_sim =[cosine_similarity(tf_idf_vals[doc_idx], tf_idf_vals[doc]) for doc in range(corpus.shape[0])]
    #odeno los resultados de mayor a menor
    cos_sim_idx = np.array(cos_sim).argsort()[::-1]
    #elimino el indice del documento que estoy comparando
    cos_sim_idx = np.delete(cos_sim_idx, np.where(cos_sim_idx == doc_idx))
    #devuelvo el corpus ordenado por similitud
    return corpus[cos_sim_idx], np.array(cos_sim)[cos_sim_idx]
    

In [16]:
#comparo el primer documento con el resto del corpus
docs, cos_sim= docs_comparator(0, corpus)
# transformo a pandas 
pd.DataFrame({'Documentos':docs, 'Similitud Coseno':cos_sim})

,Documentos,Similitud Coseno
0,martes el dia de hoy es martes,0.200342
1,martes muchas gracias,0.000000
